In [6]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import root_mean_squared_error
import pickle
import mlflow

## MLflow
### Initial setup
1. Need to run on terminal: `mlflow ui --backend-store-uri sqlite:///mlflow.db`
2. MLflow can be found on [http://127.0.0.1:5000/](http://127.0.0.1:5000/)
3. We set the tracking uri on the python script
4. We set the experiment name where all runs will be saved. It the exp doesn't exist mlflow will create one.

#### URI: Unique resource identifier

In [7]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location=('/Users/andreanicolas/Library/CloudStorage/GoogleDrive-andreanicolas91@gmail.com/My '
 'Drive/ASU_ComputerScience/MLOps_studies/02-experiment-tracking/mlruns/1'), creation_time=1716340686867, experiment_id='1', last_update_time=1716340686867, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

#### Read in taxi data using pandas.

In [8]:
def read_dataframe(filename):
    df = pd.read_parquet(filename) #read data
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime # calculate duration
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60) # convert to mins
    mask = ((df.duration >=1) & (df.duration<=60)) # select only significant data
    df = df[mask]
    categorical = ['PULocationID', 'DOLocationID'] # pickup and dropoff location
    df[categorical] = df[categorical].astype(str) # convert to string/object to prevent labeling

    return df

In [9]:
df_train = read_dataframe('../data/green_tripdata_2023-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2023-02.parquet')
len(df_train), len(df_val)

(65946, 62574)

#### dictVectorizer to generate train and validation X

In [10]:
print('combining pickup location ID and dropoff location ID...')
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

combining pickup location ID and dropoff location ID...


In [11]:
dv = DictVectorizer()
categorical = ['PU_DO'] # pickup and dropoff location
numerical = ['trip_distance'] # distance of trip

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts) # no need to fit sicne its done already for training data

#### Obtaining Y values (labels)

In [12]:
y_train = df_train['duration'].values
y_val = df_val['duration'].values

#### Fit linear regression and calculate RMSEs

In [13]:
lr = LinearRegression()
lr.fit(X_train,y_train)

y_pred_train = lr.predict(X_train)
y_pred_val   = lr.predict(X_val)

train_rmse = root_mean_squared_error(y_train,y_pred_train)
val_rmse   = root_mean_squared_error(y_val,y_pred_val)

print(f'RMSE from train data is : {train_rmse:.2f}')
print(f'RMSE from validation data is : {val_rmse:.2f}')

RMSE from train data is : 4.80
RMSE from validation data is : 6.04


#### Save model as pickle file

In [14]:
with open('../models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

## MLFlow

### Initial logs
1. We have start a run, where everytime we hit run a different run will be saved on mlflow.
2. We are saving ad-hoc parameters like train data path, validation data path, alpha
3. We are logging a goodness metric, rmse
4. We also have setup a tag with the developer name
#### LASSO model:

In [23]:
with mlflow.start_run():
    mlflow.set_tag("developer","andrea")

    # log info about dataset
    mlflow.log_param("train-data-path","../01-intro/data/green_tripdata_2023-01.parquet")
    mlflow.log_param("valid-data-path","../01-intro/data/green_tripdata_2023-02.parquet")

    #log alpha
    alpha = 0.01
    mlflow.log_param("alpha",alpha)

    lr = Lasso(alpha)
    lr.fit(X_train,y_train)

    y_pred_val   = lr.predict(X_val)

    #log metric
    val_rmse   = root_mean_squared_error(y_val,y_pred_val)
    mlflow.log_metric("rmse",val_rmse)
    print(f'RMSE from validation data is : {val_rmse:.2f}')

    # save model (pretty sure its saving linear regression one and not lasso)
    mlflow.log_artifact(local_path="../models/lin_reg.bin", artifact_path="models_pickle")

RMSE from validation data is : 8.23


### Optimizing hyperparameters

In [16]:
import xgboost as xgb #xtreme gradient boosting
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
# hyperopt is a library that using bayesian methods to find best set of hyperparameters 
#fmin - minimize obj fcn//output
#tpe - algo used to control flushing of input/output
#hp - different methods to define the search space
#status_ok - signal to send at the end of each run to tell hyperopt that method ran succesfully
#trials - will keep track of info for each run
#scope - to define range of type integer

#### Set up train and validation data to feed into obj function

In [17]:
train = xgb.DMatrix(X_train,label=y_train)
valid = xgb.DMatrix(X_val,label=y_val)

#### Define obj function

In [18]:
def objective(params):

    with mlflow.start_run():
        mlflow.set_tag("model","xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params, # modeling parameters alpha
            dtrain=train, # training data
            num_boost_round=1000,
            evals=[(valid,"validation")], # will use validation data to check for improvements
            early_stopping_rounds=50 # if there are 50 iters and no improvement happens optimization will stop
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val,y_pred)
        mlflow.log_metric("rmse",rmse)
    
    return {'loss':rmse,'status':STATUS_OK} #status ok is just signal that optimization was successful

#### Define search space

In [19]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth',4, 100, 1)), #depth of the trees, we convert to integer since the output originally is float
    'learning_rate':hp.loguniform('learning_rate',-3, 0), # exp(-3) to exp(0)... [0.05, 1]
    'reg_alpha':hp.loguniform('reg_alpha',-5, -1),
    'reg_lambda':hp.loguniform('reg_lambda',-6, -1),
    'min_child_weight':hp.loguniform('min_child_weight',-1, 3),
    'objective': 'reg:linear', # regression problem
    'seed': 42,
}

#### Optimize obj fcn by minimizing the output 

In [20]:
best_result = fmin(
    fn = objective,
    space = search_space,
    algo = tpe.suggest, # algorithm to run optimization
    max_evals = 50,
    trials= Trials() #all info will be stored here
)

[0]	validation-rmse:6.29560                           
[1]	validation-rmse:5.56526                           
[2]	validation-rmse:5.39183                           
[3]	validation-rmse:5.34047                           
[4]	validation-rmse:5.31796                           
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/andreanicolas/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [00:15:17] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[5]	validation-rmse:5.31052                           
[6]	validation-rmse:5.30097                           
[7]	validation-rmse:5.29845                           
[8]	validation-rmse:5.29336                           
[9]	validation-rmse:5.28976                           
[10]	validation-rmse:5.28616                          
[11]	validation-rmse:5.28406                          
[12]	validation-rmse:5.27899                          
[13]	validation-rmse:5.27592                          
[14]	validation-rmse:5.27334                          
[15]	validation-rmse:5.26945                          
[16]	validation-rmse:5.26775                          
[17]	validation-rmse:5.26636                          
[18]	validation-rmse:5.26443                          
[19]	validation-rmse:5.25555                          
[20]	validation-rmse:5.25380                          
[21]	validation-rmse:5.25199                          
[22]	validation-rmse:5.25035                          
[23]	valid

/Users/andreanicolas/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [00:15:21] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[2]	validation-rmse:7.32225                                                    
[3]	validation-rmse:6.89683                                                    
[4]	validation-rmse:6.55959                                                    
[5]	validation-rmse:6.29942                                                    
[6]	validation-rmse:6.08580                                                    
[7]	validation-rmse:5.92553                                                    
[8]	validation-rmse:5.79867                                                    
[9]	validation-rmse:5.70267                                                    
[10]	validation-rmse:5.62130                                                   
[11]	validation-rmse:5.56309                                                   
[12]	validation-rmse:5.51897                                                   
[13]	validation-rmse:5.48153                                                   
[14]	validation-rmse:5.44729            

...

[3]	validation-rmse:6.05393                                                     
[4]	validation-rmse:5.79478                                                     
[5]	validation-rmse:5.62758                                                     
[6]	validation-rmse:5.51869                                                     
[7]	validation-rmse:5.44839                                                     
[8]	validation-rmse:5.40001                                                     
[9]	validation-rmse:5.36832                                                     
[10]	validation-rmse:5.34267                                                    
[11]	validation-rmse:5.32428                                                    
[12]	validation-rmse:5.31136                                                    
[13]	validation-rmse:5.30147                                                    
[14]	validation-rmse:5.29393                                                    
[15]	validation-rmse:5.28946

/Users/andreanicolas/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [00:32:04] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[1]	validation-rmse:8.30645                                                     
[2]	validation-rmse:7.88738                                                     
[3]	validation-rmse:7.52232                                                     
[4]	validation-rmse:7.20336                                                     
[5]	validation-rmse:6.92568                                                     
[6]	validation-rmse:6.68542                                                     
[7]	validation-rmse:6.47770                                                     
[8]	validation-rmse:6.29906                                                     
[9]	validation-rmse:6.14404                                                     
[10]	validation-rmse:6.01313                                                    
[11]	validation-rmse:5.89997                                                    
[12]	validation-rmse:5.80338                                                    
[13]	validation-rmse:5.72097

/Users/andreanicolas/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [00:32:31] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.67956                                                     
[1]	validation-rmse:8.12891                                                     
[2]	validation-rmse:7.65972                                                     
[3]	validation-rmse:7.26074                                                     
[4]	validation-rmse:6.92170                                                     
[5]	validation-rmse:6.63793                                                     
[6]	validation-rmse:6.39691                                                     
[7]	validation-rmse:6.19777                                                     
[8]	validation-rmse:6.03140                                                     
[9]	validation-rmse:5.89492                                                     
[10]	validation-rmse:5.77707                                                    
[11]	validation-rmse:5.68443                                                    
[12]	validation-rmse:5.60400

#### Verifying that best parameters do the trick

In [21]:
best_params = {
'learning_rate': 0.11405256196340997,
'max_depth':36,
'min_child_weight':3.711408284036696,
'objective':'reg:linear',
'reg_alpha':0.2047112668129924,
'reg_lambda':0.01260039738825428,
'seed':42
}

### NO Autologging
Can do what was done previously, with mlflow.start_run etc etc...

In [25]:
mlflow.xgboost.autolog(disable=True)

In [27]:
with mlflow.start_run():
    # log tag and parameters
    mlflow.set_tag("model","xgboost")
    mlflow.log_params(best_params)
    # train model
    booster = xgb.train(
        params=best_params, # modeling parameters alpha
        dtrain=train, # training data
        num_boost_round=1000,
        evals=[(valid,"validation")], # will use validation data to check for improvements
        early_stopping_rounds=50 # if there are 50 iters and no improvement happens optimization will stop
    )
    #calculate metric and log
    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val,y_pred)
    mlflow.log_metric("rmse",rmse)

    # save preprocessor as pickle file
    with open("../models/preprocessor.b","wb") as f_out:
        pickle.dump(dv, f_out)

    # log the preprocessors as artifacts
    mlflow.log_artifact("../models/preprocessor.b",artifact_path="preprocessor")

    # log model as artifact
    # NOTE: this first ran with autologging on, because we had run it before
    # we would need to disable autologging to prevent duplicate artifacts
    mlflow.xgboost.log_model(booster,artifact_path="models_mlflow")

[0]	validation-rmse:8.62314


/Users/andreanicolas/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [12:20:54] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[1]	validation-rmse:8.03055
[2]	validation-rmse:7.53232
[3]	validation-rmse:7.11643
[4]	validation-rmse:6.76941
[5]	validation-rmse:6.48416
[6]	validation-rmse:6.24883
[7]	validation-rmse:6.05554
[8]	validation-rmse:5.89646
[9]	validation-rmse:5.76851
[10]	validation-rmse:5.66179
[11]	validation-rmse:5.57584
[12]	validation-rmse:5.50703
[13]	validation-rmse:5.45182
[14]	validation-rmse:5.40683
[15]	validation-rmse:5.36931
[16]	validation-rmse:5.33743
[17]	validation-rmse:5.31110
[18]	validation-rmse:5.29017
[19]	validation-rmse:5.27248
[20]	validation-rmse:5.25816
[21]	validation-rmse:5.24655
[22]	validation-rmse:5.23587
[23]	validation-rmse:5.22666
[24]	validation-rmse:5.22105
[25]	validation-rmse:5.21497
[26]	validation-rmse:5.20958
[27]	validation-rmse:5.20458
[28]	validation-rmse:5.19995
[29]	validation-rmse:5.19746
[30]	validation-rmse:5.19466
[31]	validation-rmse:5.19181
[32]	validation-rmse:5.18891
[33]	validation-rmse:5.18613
[34]	validation-rmse:5.18475
[35]	validation-rmse:5.

/Users/andreanicolas/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [12:21:07] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
/Users/andreanicolas/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/Users/andreanicolas/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distuti

### Loging models in MLflow
Two options:
- log model as an artifact
  `mlflow.log_artifact("mymodel",artifact_path="models"/)`
- log model using log_model
  `mlflow.<framework>.log_model(model,artifact_path="models")`

### With Autologging
Automatic logging allows you to log metrics, parameters, and models without explicit statements. 

all you need is autolog()

Only works is you use the following libraries:
- Scikit-learn
- Tensorflow and keras
- XGBoost
- Spark
- Pytorch
- Gluon
- LightGBM
- Statsmodels
- Fastai

You can find more information on the mlflow documentation [here](https://mlflow.org/docs/latest/tracking/autolog.html#automatic-logging).

In [22]:
mlflow.xgboost.autolog()

# train model only
booster = xgb.train(
    params=best_params, # modeling parameters alpha
    dtrain=train, # training data
    num_boost_round=1000,
    evals=[(valid,"validation")], # will use validation data to check for improvements
    early_stopping_rounds=50 # if there are 50 iters and no improvement happens optimization will stop
)


2024/05/22 00:58:04 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '55e6319f043c4d438ab65fff41edf3fe', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[0]	validation-rmse:8.62314


/Users/andreanicolas/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [00:58:05] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[1]	validation-rmse:8.03055
[2]	validation-rmse:7.53232
[3]	validation-rmse:7.11643
[4]	validation-rmse:6.76941
[5]	validation-rmse:6.48416
[6]	validation-rmse:6.24883
[7]	validation-rmse:6.05554
[8]	validation-rmse:5.89646
[9]	validation-rmse:5.76851
[10]	validation-rmse:5.66179
[11]	validation-rmse:5.57584
[12]	validation-rmse:5.50703
[13]	validation-rmse:5.45182
[14]	validation-rmse:5.40683
[15]	validation-rmse:5.36931
[16]	validation-rmse:5.33743
[17]	validation-rmse:5.31110
[18]	validation-rmse:5.29017
[19]	validation-rmse:5.27248
[20]	validation-rmse:5.25816
[21]	validation-rmse:5.24655
[22]	validation-rmse:5.23587
[23]	validation-rmse:5.22666
[24]	validation-rmse:5.22105
[25]	validation-rmse:5.21497
[26]	validation-rmse:5.20958
[27]	validation-rmse:5.20458
[28]	validation-rmse:5.19995
[29]	validation-rmse:5.19746
[30]	validation-rmse:5.19466
[31]	validation-rmse:5.19181
[32]	validation-rmse:5.18891
[33]	validation-rmse:5.18613
[34]	validation-rmse:5.18475
[35]	validation-rmse:5.

2024/05/22 00:58:22 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/05/22 00:58:22 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/andreanicolas/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [00:58:22] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."


### Reading model from MLflow

In [28]:
#import mlflow
logged_model = 'runs:/0b983bbfdc7148a3951d7bce9c997594/models_mlflow'

# Load model as a PyFuncModel.
py_loaded_model = mlflow.pyfunc.load_model(logged_model)

# load model as XGBoost model
xgb_loaded_model = mlflow.xgboost.load_model(logged_model)

/Users/andreanicolas/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [12:26:52] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [30]:
y_pred_xgb = xgb_loaded_model.predict(valid)
y_pred_xgb[:10]

array([22.17077  , 21.226362 , 24.710638 , 18.949577 , 13.896604 ,
       13.295256 , 12.657442 ,  7.4791665, 13.631114 , 11.685048 ],
      dtype=float32)